# ทดสอบการเข้ารหัสแบบกุญแจสาธารณะ (pgp_pub_encrypt/pgp_pub_decrypt) ด้วย ORM

โน้ตบุ๊กนี้สาธิตการใช้งาน `pgcrypto` แบบกุญแจสาธารณะร่วมกับ ORM (SQLAlchemy) โดย
- สร้างกุญแจสาธารณะ/กุญแจส่วนตัว (OpenPGP) ด้วยไลบรารี `pgpy` ใน Python
- บันทึกข้อมูลด้วยการเข้ารหัสที่ฝั่งฐานข้อมูล: `pgp_pub_encrypt(data, public_key)`
- อ่านข้อมูลด้วยการถอดรหัสที่ฝั่งฐานข้อมูล: `pgp_pub_decrypt(cipher, private_key[, passphrase])`

คำเตือน: ตัวอย่างนี้สร้างกุญแจใหม่ทุกครั้งเพื่อการสาธิต — ข้อมูลที่เข้ารหัสด้วยกุญแจชุดหนึ่งจะถอดไม่ได้หากสูญเสียกุญแจส่วนตัวของชุดนั้น


## เตรียมสภาพแวดล้อม
- รัน `docker-compose up -d` เพื่อเริ่มฐานข้อมูล (เปิด `pgcrypto` อัตโนมัติ)
- แนะนำให้เปิดใช้งาน venv และติดตั้งไลบรารีใน `requirements.txt`
- หากยังไม่ได้ติดตั้ง สามารถใช้เซลล์ pip ด้านล่างนี้ (ชั่วคราว)

In [ ]:
# ติดตั้งไลบรารีที่จำเป็น (ข้ามได้ถ้าติดตั้งแล้ว)
# !pip install -q "psycopg[binary]>=3.1" SQLAlchemy>=2.0 pandas pgpy>=0.5


## สร้างกุญแจสาธารณะ/ส่วนตัว (OpenPGP) ด้วย pgpy
- เพื่อการสาธิตเราจะสร้างกุญแจใหม่ในหน่วยความจำทุกครั้ง
- หากต้องการใช้กุญแจถาวร ควรจัดเก็บไว้ (เช่น ไฟล์/Secret Manager) และกำหนดรหัสผ่านที่รัดกุม

In [ ]:
import os, datetime
import pandas as pd
from sqlalchemy import create_engine, Text, LargeBinary, func, text, select, type_coerce
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, Session
from sqlalchemy.types import TypeDecorator

import pgpy
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm

# สร้างกุญแจใหม่ (เพื่อสาธิต)
uid = pgpy.PGPUID.new('Demo User', comment='PGP Demo', email='demo@example.com')
key = pgpy.PGPKey.new(PubKeyAlgorithm.RSAEncryptOrSign, 2048)
key.add_uid(uid,
           usage={KeyFlags.EncryptCommunications, KeyFlags.EncryptStorage},
           hashes=[HashAlgorithm.SHA256],
           ciphers=[SymmetricKeyAlgorithm.AES256],
           compression=[CompressionAlgorithm.ZLIB])

# ป้องกันกุญแจส่วนตัวด้วยรหัสผ่านสำหรับสาธิต
PRIVATE_PASSPHRASE = os.getenv('PG_PRIVATE_PASSPHRASE', 'demo-private-passphrase')
key.protect(PRIVATE_PASSPHRASE, SymmetricKeyAlgorithm.AES256, HashAlgorithm.SHA256)

PUBLIC_KEY_TEXT = str(key.pubkey)
PRIVATE_KEY_TEXT = str(key)
print('สร้างคู่กุญแจเสร็จสิ้น')


## ตั้งค่าการเชื่อมต่อ และประกาศโมเดล ORM พร้อมชนิดข้อมูลเข้ารหัสแบบกุญแจสาธารณะ

In [ ]:
PG_HOST = os.getenv('PG_HOST', 'localhost')
PG_PORT = int(os.getenv('PG_PORT', '5432'))
PG_DB   = os.getenv('PG_DB', 'encdb')
PG_USER = os.getenv('PG_USER', 'encuser')
PG_PWD  = os.getenv('PG_PASSWORD', 'encpass')

engine = create_engine(
    f'postgresql+psycopg://{PG_USER}:{PG_PWD}@{PG_HOST}:{PG_PORT}/{PG_DB}',
    future=True,
)

# เปิดใช้ส่วนขยาย pgcrypto หากยังไม่ได้เปิด
with engine.begin() as conn:
    conn.execute(text('CREATE EXTENSION IF NOT EXISTS pgcrypto;'))

class Base(DeclarativeBase):
    pass

class PgPubEncryptedString(TypeDecorator):
    impl = LargeBinary  # เก็บในฐานข้อมูลเป็น bytea
    cache_ok = True

    def __init__(self, public_key_text: str, private_key_text: str, private_passphrase: str | None = None):
        super().__init__()
        self.public_key_text = public_key_text
        self.private_key_text = private_key_text
        self.private_passphrase = private_passphrase

    # ก่อนบันทึก: เข้ารหัสฝั่งฐานข้อมูลด้วยกุญแจสาธารณะ
    def bind_expression(self, bindvalue):
        return func.pgp_pub_encrypt(type_coerce(bindvalue, Text()), self.public_key_text)

    # ตอนอ่าน: ถอดรหัสฝั่งฐานข้อมูลด้วยกุญแจส่วนตัว (และ passphrase ถ้ามี)
    def column_expression(self, col):
        if self.private_passphrase:
            return type_coerce(func.pgp_pub_decrypt(col, self.private_key_text, self.private_passphrase), Text())
        else:
            return type_coerce(func.pgp_pub_decrypt(col, self.private_key_text), Text())

class CustomerSecretPubORM(Base):
    __tablename__ = 'customer_secret_pub_orm'
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    full_name: Mapped[str] = mapped_column(Text, nullable=False)
    national_id: Mapped[str] = mapped_column(
        PgPubEncryptedString(PUBLIC_KEY_TEXT, PRIVATE_KEY_TEXT, PRIVATE_PASSPHRASE),
        nullable=False
    )

# สร้างตารางใหม่เพื่อการทดสอบ (ลบทิ้งก่อน)
with engine.begin() as conn:
    conn.execute(text('DROP TABLE IF EXISTS customer_secret_pub_orm'))
Base.metadata.create_all(engine)
print('สร้างตาราง customer_secret_pub_orm สำเร็จ')


## แทรกข้อมูล (จะถูกเข้ารหัสด้วยกุญแจสาธารณะโดยอัตโนมัติ)

In [ ]:
samples = [
    ('สมชาย ใจดี', '1101700200011'),
    ('สมหญิง แข็งแรง', '1101700200022'),
    ('สายฝน สายชล', '1101700200033'),
]

with Session(engine) as session:
    session.add_all([CustomerSecretPubORM(full_name=n, national_id=i) for n, i in samples])
    session.commit()
print('เพิ่มข้อมูลสำเร็จ (ถูกเข้ารหัสก่อนบันทึก)')


## อ่านข้อมูลแบบเข้ารหัส (ยังไม่ถอดรหัส)

In [ ]:
with Session(engine) as session:
    result = session.execute(
        select(
            CustomerSecretPubORM.id,
            CustomerSecretPubORM.full_name,
            CustomerSecretPubORM.__table__.c.national_id.label('national_id_encrypted'),
            func.encode(CustomerSecretPubORM.__table__.c.national_id, 'hex').label('ct_hex'),
            func.octet_length(CustomerSecretPubORM.__table__.c.national_id).label('ct_bytes')
        ).order_by(CustomerSecretPubORM.id)
    ).all()
pd.DataFrame(result, columns=['id', 'full_name', 'national_id_encrypted', 'ct_hex', 'ct_bytes'])


## อ่านข้อมูลแบบถอดรหัส (ผ่าน column_expression)

In [ ]:
with Session(engine) as session:
    result = session.execute(
        select(CustomerSecretPubORM.id, CustomerSecretPubORM.full_name, CustomerSecretPubORM.national_id)
        .order_by(CustomerSecretPubORM.id)
    ).all()
pd.DataFrame(result, columns=['id', 'full_name', 'national_id'])


### หมายเหตุ
- ตัวอย่างนี้สร้างกุญแจทุกครั้ง ข้อมูลที่เขียนด้วยกุญแจชุดนี้จะถอดไม่ได้หากเปลี่ยนกุญแจ
- หากต้องการใช้งานจริง ให้สร้าง/เก็บกุญแจถาวร และกำหนดสิทธิ์เข้าถึงอย่างเหมาะสม
- `pgp_pub_encrypt`/`pgp_pub_decrypt` เก็บข้อมูลเป็น `bytea` ในตาราง และไม่เหมาะกับการทำ index โดยตรง
